##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 전문가를 위한 TensorFlow 2 빠른 시작

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/advanced"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소그 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운론드하기</a></td>
</table>

이것은 [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) 노트북 파일입니다. Python 프로그램은 브라우저에서 직접 실행되므로 TensorFlow를 배우고 사용하기에 좋습니다. 이 튜토리얼을 따르려면 이 페이지 상단에 있는 버튼을 클릭하여 Google Colab에서 노트북을 실행하세요.

1. Colab에서 Python 런타임에 연결: 메뉴 막대의 오른쪽 상단에서 *CONNECT*를 선택합니다.
2. 노트북의 모든 코드 셀(cell)을 실행하세요: *Runtime* &gt; *Run all*을 선택하세요.

TensorFlow 2를 다운로드하여 설치합니다. TensorFlow를 프로그램으로 가져옵니다.

참고: TensorFlow 2 패키지를 설치하려면 `pip`를 업그레이드하세요. 자세한 내용은 [설치 가이드](https://www.tensorflow.org/install)를 참조하세요.

TensorFlow를 프로그램으로 가져옵니다.

In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

TensorFlow version: 2.15.0


[MNIST](http://yann.lecun.com/exdb/mnist/) 데이터셋을 로드하여 준비합니다.

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

11490434/11490434 [==============================] - 2s 0us/step


`tf.data`를 사용하여 데이터세트를 섞고 배치를 만듭니다:

In [4]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Keras <a>model subclassing API</a>를 사용하는 <code>tf.keras</code> 모델을 다음과 같이 빌드합니다.

In [5]:
class MyModel(Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

훈련에 필요한 옵티마이저(optimizer)와 손실 함수를 선택합니다:

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

모델의 손실과 성능을 측정할 지표를 선택합니다. 에포크가 진행되는 동안 수집된 측정 지표를 바탕으로 최종 결과를 출력합니다.

In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

`tf.GradientTape`을 사용하여 모델을 훈련합니다:

In [8]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

모델 테스트하기:

In [9]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [10]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.12936876714229584, Accuracy: 96.08000183105469, Test Loss: 0.06655524671077728, Test Accuracy: 97.8499984741211
Epoch 2, Loss: 0.04102810099720955, Accuracy: 98.75166320800781, Test Loss: 0.049070458859205246, Test Accuracy: 98.31999969482422
Epoch 3, Loss: 0.02132483385503292, Accuracy: 99.2933349609375, Test Loss: 0.051658909767866135, Test Accuracy: 98.52999877929688
Epoch 4, Loss: 0.013333531096577644, Accuracy: 99.5616683959961, Test Loss: 0.061985574662685394, Test Accuracy: 98.31999969482422
Epoch 5, Loss: 0.006873783189803362, Accuracy: 99.80000305175781, Test Loss: 0.057833991944789886, Test Accuracy: 98.56999969482422


훈련된 이미지 분류기는 이 데이터셋에서 약 98%의 정확도를 달성합니다. 더 자세한 내용은 [TensorFlow 튜토리얼을 참고하세요](https://www.tensorflow.org/tutorials).

In [11]:

probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

In [17]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image

# Load and preprocess the image
img_path = "/content/d993cffadff907ffc455e6cd1dfc7863.jpg"
img = image.load_img(img_path, target_size=(28,28), color_mode='grayscale') # Load as grayscale
img_array = image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create a batch

# Make the prediction
predictions = probability_model.predict(img_array)

# Process the predictions (e.g., print the class with the highest probability)
print(predictions)

1/1 [==============================] - 0s 173ms/step
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]


In [19]:
predictions = model.predict(img_array)
print(predictions)

1/1 [==============================] - 0s 34ms/step
[[  942.5619  -2134.1873  -2041.1707   -232.47217 -1152.0706   1836.1155
   -230.455    -796.3023    415.35504   185.96527]]
